# Mt. Edgecumbe volcanic InSAR time series analysis with HyP3 and MintPy

In this notebook, you will learn how to perform a time series InSAR analysis using the popular Small BAseline Subset (SBAS) technique. SBAS is well suited for monitoring slow, consistent deformation of the earth's surface caused by processes such as slow landslides, subsidence, fault creep and volcanic inflation. In this tutorial we will focus on a volcanic inflation use case at [Mt. Edgecumbe, Alaska](https://en.wikipedia.org/wiki/Mount_Edgecumbe_(Alaska)).

In April 2022, a seismic swarm near Mt. Edgecumbe in southeast Alaska suggested there could be renewed activity at the volcano. However, since the last sign of activity at Mt. Edgecumbe was about 800 years ago, there was no monitoring equipment installed at the volcano. This made it difficult to determine if the seismic swarm was unrelated to volcanic activity, or if the volcano was returning to an active state. Luckily, the team at the Alaska Volcano Observatory (AVO) included experienced InSAR users, and they were able use a time series InSAR analysis to show that volcanic inflation had begun at the site in 2018. This analysis took much less time than more traditional methods, and was critical for providing timely information about the status of the volcano to the neighboring city of Sitka, Alaska.

This notebook will show you how to perform the InSAR time series analysis that the AVO team used to reach their conclusions. It follows the methodology detailed in their recent paper [(Grapenthin et al., 2022)](https://doi.org/10.1029/2022GL099464), which utilizes On Demand InSAR products from the Alaska Satellite Facility (ASF) and the MintPy time series analysis software. 

In this notebook we will:

<br></br>1. Use the [HyP3 Python SDK](https://hyp3-docs.asf.alaska.edu/using/sdk/) to:
   - Request On Demand InSAR products from ASF HyP3
   - Download the InSAR products when they are done processing
<br></br>2. Use [MintPy](https://mintpy.readthedocs.io/en/latest/) and various Python utilities to:
   - Download staged data that we will use for the time series (check out the `data_prep.py` script to see how the data was prepared)
   - Assess the quality of the interferogram network we will use for the analysis
   - Assess whether the data quality is suitable for a time series analysis
   - Perform SBAS time series analyses for three InSAR stacks over Mt. Edgecumbe
   - Demonstrate how to view the results of a time series analysis
   - Reproduce the main InSAR figure from the AVO team's recent publication [(Figure 2 from Grapenthin et al., 2022)](https://doi.org/10.1029/2022GL099464)
---

**Note:** This notebook uses staged data to ensure that there is adequate time to step through the InSAR time series workflow and discuss the analysis methods. It also assumes that you have some familiarity with InSAR processing already. If you're new to InSAR and working with ASF's on-demand services, you may find the following resources useful:
1. ASF's [InSAR On Demand StoryMap](https://storymaps.arcgis.com/stories/68a8a3253900411185ae9eb6bb5283d3)
2. The `data_prep.py` script in the same directory as this notebook
3. [OpenSARlab](https://opensarlab-docs.asf.alaska.edu/)'s highly detailed walkthrough of preparing ASF's On-Demand (HyP3) data for MintPy using these notebooks:
    - [Prepare a HyP3 InSAR Stack for MintPy](https://nbviewer.org/github/ASFOpenSARlab/opensarlab-notebooks/blob/master/SAR_Training/English/Master/Prepare_HyP3_InSAR_Stack_for_MintPy.ipynb)
    - [MintPy Time-series Analysis](https://nbviewer.org/github/ASFOpenSARlab/opensarlab-notebooks/blob/master/SAR_Training/English/Master/MintPy_Time_Series_From_Prepared_Data_Stack.ipynb)
---
This notebook and the ones listed above assume that you're working in OpenSARlab. However, you can also run these 
notebooks outside of OpenSARlab by creating [this conda environment](https://github.com/ASFOpenSARlab/opensarlab-envs/blob/main/Environment_Configs/insar_analysis_env.yml) and launching these notebooks from within this environment.

## 0. Initial Setup

To run this notebook, you'll need to be in the `insar_analysis` conda environment within OpenSARLab.

Alternatively, you can set up your own environment by running these commands in your shell (you'll need to have [conda](https://docs.conda.io/projects/continuumio-conda/en/latest/user-guide/install/index.html) installed):
```shell
curl -OL https://raw.githubusercontent.com/ASFOpenSARlab/opensarlab-envs/main/Environment_Configs/insar_analysis_env.yml
conda env create -f insar_analysis_env.yml
```
Then launch this notebook from the new environment:
```shell
conda activate insar_analysis
jupyter lab igarss_mtedgecumbe_ts_analysis.ipynb
```

Once you have completed the setup for one of these two environments, you are ready to start working with the data.

## 1. Request On Demand InSAR products from ASF HyP3

A major step towards working with SAR data at scale is learning how to request and download data programmatically. Accomplishing these tasks via code makes it much easier to request large quantities of data and to make similar requests in the future.

To request the generation of an Interferometric Synthetic Aperture Radar (InSAR) product from ASF, you can follow the general steps outlined in the cells below. For this example, we'll be requesting the generation of an interferogram that shows inflation-related displacement at Mt. Edgecumbe:

1. **Create an account**: If you don't already have NASA Earthdata Login credentials, create a [NASA Earthdata login](https://urs.earthdata.nasa.gov/) profile. This will allow you connect to ASF HyP3 via the Python SDK. Running the cell below will prompt you for your Earthdata username and password

In [ ]:
import hyp3_sdk as sdk
hyp3 = sdk.HyP3(prompt=True)

2. **Select your data**: Use ASF's [Vertex](https://search.asf.alaska.edu/) data search portal to find Sentinel-1 scenes you would like to process. For this tutorial, we will use the following scene pair:

In [ ]:
reference = 'S1B_IW_SLC__1SDV_20180628T151540_20180628T151607_011575_015476_4673'
secondary = 'S1B_IW_SLC__1SDV_20190705T151547_20190705T151614_017000_01FFC4_D62F'

3. **Define your processing parameters**: Determine the specific processing parameters for your InSAR product. This includes selecting the number of looks (which determines the pixel spacing of the output product), and whether to apply a water mask before phase unwrapping. If you are using the InSAR products for MintPy analysis, you will also need to include the DEM and look vector products with your output product. You can read more about the available parameters within the [HyP3 SDK documentation](https://hyp3-docs.asf.alaska.edu/using/sdk_api/#hyp3_sdk.hyp3.HyP3.submit_insar_job).

In [ ]:
looks='20x4'
include_dem=True
include_look_vectors=True
apply_water_mask=True

4. **Submit a processing request**: Once you have defined your parameters, use the HyP3 SDK's [`submit_insar_job`](https://hyp3-docs.asf.alaska.edu/using/sdk_api/#hyp3_sdk.hyp3.HyP3.submit_insar_job) function to submit a processing request. You can use the `project_name` name argument to group sets of requests together under one name so that you can easily look them up later.

In [ ]:
import hyp3_sdk as sdk
project_name = 'edgecumbe_example'
jobs = sdk.Batch()
jobs += hyp3.submit_insar_job(reference, secondary, name=project_name, 
                              looks=looks, include_dem=include_dem, include_look_vectors=include_look_vectors, apply_water_mask=apply_water_mask)

5. **Monitor the processing status**: After submitting your request, ASF will process your data using HyP3, and output an InSAR product. You can monitor the status of your request by calling the [`watch`](https://hyp3-docs.asf.alaska.edu/using/sdk_api/#hyp3_sdk.hyp3.HyP3.watch) HyP3 SDK method.

In [ ]:
jobs = hyp3.watch(jobs)

6. **Access and download the InSAR product**: Once processing is complete, your InSAR job status will be updated to `SUCCEEDED` and your data will be ready to download. You can download the data by searching for jobs using your project name and the `SUCCEEDED` status code...

In [ ]:
jobs = hyp3.find_jobs(name=project_name, status_code = 'SUCCEEDED')

...then using the `download_files` method to download the data. The HyP3 SDK also includes an `extract_zipped_product` utility that you can use to unzip the files.

In [ ]:
import hyp3_sdk as sdk
insar_products = jobs.download_files('.')
insar_products = [sdk.util.extract_zipped_product(ii) for ii in insar_products]

Now that we've requested and downloaded the interferogram, let's take a look at it. Here we plot the unwrapped interferogram overlaid on the DEM used for processing. This interferogram shows the inflation deformation signal that Grapenthin et al. originally discovered. 

In [ ]:
%matplotlib inline
import numpy as np
from osgeo import gdal
import matplotlib
import matplotlib.pyplot as plt

unwrapped_file = 'S1BB_20180628T151540_20190705T151547_VVP372_INT80_G_weF_A722/S1BB_20180628T151540_20190705T151547_VVP372_INT80_G_weF_A722_unw_phase.tif'
amp_file = 'S1BB_20180628T151540_20190705T151547_VVP372_INT80_G_weF_A722/S1BB_20180628T151540_20190705T151547_VVP372_INT80_G_weF_A722_amp.tif'

ds = gdal.Open(unwrapped_file)
unwrapped = np.ma.masked_equal(ds.GetRasterBand(1).ReadAsArray(), 0)
del ds

ds = gdal.Open(amp_file)
amp = np.ma.masked_equal(ds.GetRasterBand(1).ReadAsArray(), 0)
del ds

f, ax = plt.subplots(1,1, figsize=(6,6))
ax.imshow(unwrapped[1250:1575, 700:1000], alpha=0.75, cmap='jet')
ax.imshow(amp[1250:1575, 700:1000], alpha=0.4, cmap='Greys', norm=matplotlib.colors.LogNorm())
ax.set_title('Unwrapped Interferogram')

plt.setp(plt.gcf().get_axes(), xticks=[], yticks=[])
plt.tight_layout()

We have now gone through the process of requesting and downloading a single interferogram, but time series InSAR workflows routinely make use of hundreds of interferograms (Grapenthin et al., 2022 used more than a thousand!). We don't have time to request and download this many interferograms during this workshop, so instead we've pre-generated and prepped some InSAR products for you to work with. Think of it as your very own InSAR cooking show!

## 3. Time-series Analysis with MintPy

For this analyis, we are using InSAR pair lists sourced from the [supplementary information](https://zenodo.org/record/7151431) published by Grapenthin et al.. As mentioned in the section above, we do not have enough time in this workshop to download and create all of the InSAR stacks, so we have done this for you.

To prep the data we have:

1. Parsed Grapenthin et al.'s [supplementary information](https://zenodo.org/record/7151431) file to grab a list of all the InSAR pairs we want to process
2. Grouped these InSAR pairs into three data stacks which each have a unique orbit direction, path, and frame combination, and submitted each stack for processing via HyP3. We've also submitted a fourth stack that will help us explore the effect of water masking
3. Downloaded the resulting interferogram files
4. Clipped all input data to an area of interest where all the interferograms have valid data (See section 3.1 of our [basic MintPy notebook](https://nbviewer.org/github/ASFHyP3/hyp3-docs/blob/main/docs/tutorials/hyp3_insar_stack_for_ts_analysis.ipynb))
5. Loaded a subset of the data surrounding Mt. Edgecumbe into MintPy by running MintPy's initial `load_data` step

Take a look at the `prep_data.py` script in the same directory as this notebook for the details on how we prepared the data.

### 3.1 Download the Staged Data

Let's go ahead and download the staged data from Amazon Web Services cloud storage solution, the Simple Storage Service (S3). This cell will download the data from S3, then unzip it into your working directory.

In [ ]:
import boto3
import os
import zipfile
from pathlib import Path
from botocore import UNSIGNED
from botocore.client import Config

s3_client = boto3.client('s3', config=Config(signature_version=UNSIGNED))
files = ['ascending_50.zip', 'ascending_79.zip', 'descending_174.zip', 'descending_174_nomask.zip']

for file in files:
    download_path = Path(file)
    s3_client.download_file('ffwilliams2-shenanigans', f'igarss_2023/{file}', download_path)
    with zipfile.ZipFile(download_path, 'r') as zip_ref:
        zip_ref.extractall(download_path.stem)
    os.unlink(download_path)

### 3.2 MintPy Background

The main interface for MintPy is the command line interface (CLI) tool `smallbaselineApp.py`, which you can call from this notebook or in your terminal. Whenever you begin working with a new CLI tool, it's a good idea to read the introductory documentation for the tool. This will save a ton of headaches in the long run! MintPy's documentation can be found on its [GitHub page](https://github.com/insarlab/MintPy), or by calling the tool with the `--help` flag (see below). Most Python CLI tools have a `--help` option, so get in the habit of using it whenever you begin working with a new tool.

In [ ]:
!smallbaselineApp.py --help

MintPy has a ton of configuration options that we will use to produce the best analysis that we can, and to replicate the workflow of Grapenthin et al.. We'll be using the defaults for many of MintPy's settings (you can see all of MintPy's defaults by running the command `smallbaselineApp.py -H`), but we will set some key settings ourselves. Run the command below to view the settings for one of our InSAR stacks. Other than the data loading options, we use the same settings for all stacks.

In [ ]:
!cat descending_174/mintpy_config.txt

The settings can be broken down into six groups: input data options, geographic options, inteferogram network options, unwrapping error correction options, tropospheric correction options, and topographic residual correction options:

1. **Input Data Options:**
    - The settings in the first three sections (processor, interferogram datasets, and geometry datasets) tell MintPy which InSAR processor we used to create our data, and where to find various datasets it needs for processing.
2. **Geographic Options:**
    -  The next section gives MintPy some geographic information. It tells MintPy to only load a subset of the data surrounding Mt. Edgecumbe, and to use the point `6330696, 456350` as the reference point for the time series velocity calculation. Both of these options are specified in the geographic projection of the input data, which in this case is UTM zone 8N (EPSG:32608).
3. **Interferogram Network Options:**
    - An important addage to remember when conducting InSAR analyses is *"garbage in, garbage out"*. If you include poor quality (highly-decorrelated) interferograms in your analysis, they will lead to poor results that are difficult to interpret. For this reason, we exclude any interferograms that have an average coherence less than `0.7`. **THIS IS THE MOST IMPORTANT PARAMETER YOU WILL SET WHEN USING MINTPY!!!** Take the time to make sure this value is right for your use case. In addition, MintPy gives you the option to remove individual problematic interferograms using the `mintpy.network.excludeIfgIndex` option. In most cases it's worth taking the time to go through each interferogram individually and remove any that have obvious decorrelation or unwrapping issues.
4. **Unwrapping Error Correction Options:**
    - Unwrapping error correction uses phase closure trends to reduce the number of unwrapping errors in the input data (read more [here](https://doi.org/10.1016/j.cageo.2019.104331)) and is turned on by default. Unfortunately HyP3 is not compatible with this step because it does not produce some of the needed input data, so we will be skipping this correction.
5. **Tropospheric Phase Correction Options:**
    - SAR signals propogate through the troposphere at slightly different rates depending on atmospheric conditions, which can lead to troposphere-related signasl in interferograms that we need to correct. You can learn more about this error source [here](https://doi.org/10.1016/j.earscirev.2019.03.008). Most troposphere correction algorithms utilize external atmospheric data to predict the error caused by this effect, then correct for it. In this case, we're using the [PyAPS](https://github.com/insarlab/PyAPS) Python package in conjunction with [ERA5](https://www.ecmwf.int/en/forecasts/dataset/ecmwf-reanalysis-v5) data to perform this correction. 
6. **Topographic Residual Correction Options:**
    - Errors in the DEMs used for InSAR processing can lead to errors that are proportional to the perpendicular baseline of the InSAR pair. This is another effect that is important to correct for. MintPy uses the method proposed by [Heresh and Amelung, 2013](https://doi.org/10.1109/TGRS.2012.2227761) to perform this correction.

### 3.3 Run MintPy Analysis

Now that we've covered the basics of our MintPy configuration, it's time to perform our analysis! As mentioned, above we will be processing data from four stacks, so you'll need to run the section of code starting at section 3.3 (here) to just before section 4 four times. Once for each data stack. Make sure to run each stack in order by modifying the cell below.

In [ ]:
# Make sure you run MintPy for all four stacks
stacks = ['ascending_50', 'ascending_79', 'descending_174', 'descending_174_nomask']
stack = stacks[1]

#### 3.3.1 Check Network

As we mentioned before, selecting the right interferogram network is the most important way to ensure a high quality time series InSAR analysis. Because of this, we're going to look at various ways you can analyze the quality of your interferogram network.

##### 3.3.1.1 Network Connectivity

The first thing to check is that you have a fully connected network (e.g., every date in your analysis window is covered by at least one interferogram). This can be a challenge to ensure sometimes due to seasonal decorrelation issues. In snowy areas like Mt. Edgecumbe, the presence of snow on the ground introduces significant decorrelation effects that hinder the accurate measurement of surface displacements. As a result, interferograms generated during winter periods tend to be unreliable. To overcome this challenge, it becomes necessary to exclude interferograms from the winter season. However, excluding winter interferograms leaves a data gap that can hamper your connectivity. To address this issue, you can create longer baseline interferograms, which span a longer period of time and cover the missing winter period. By including a few inteferograms with long temporal baselines, you can ensure connectivity while also not including any highly decorrelated interferograms. See the graph below to see the network that Grapenthin et al. used.

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.collections import LineCollection
from mintpy.utils.network import get_date12_list

pair_list = pd.DataFrame([pair.split('_') for pair in get_date12_list(f'{stack}/inputs/ifgramStack.h5')], columns=['date1', 'date2'])
pair_list['date1'] = pd.to_datetime(pair_list['date1'])
pair_list['date2'] = pd.to_datetime(pair_list['date2'])

# subplot 1 data
lines = []
bridges = []
for i, row in pair_list.iterrows():
    point1 = [mdates.date2num(row['date1']), i]
    point2 = [mdates.date2num(row['date2']), i]
    if point2[0] - point1[0] > 48:
        bridges.append([point1, point2])
    else:
        lines.append([point1, point2])

# subplot 2 data
date1_number = mdates.date2num(pair_list.date1)
date2_number = mdates.date2num(pair_list.date2)
dates = np.concatenate((date1_number, date2_number))
min_date = np.min(dates)
max_date = np.max(dates)

date_range = np.arange(np.min(dates), np.max(dates))
coverage = np.zeros(int(np.max(dates) - np.min(dates)))
for ifg_date1, ifg_date2 in zip(date1_number, date2_number):
    ifg_coverage = np.arange(ifg_date1 - min_date, ifg_date2 - min_date, dtype=int)
    coverage[ifg_coverage] += 1


f, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 8), sharex=True)
line_segments = LineCollection(lines, array=range(len(lines)), linewidths=1, cmap='gist_rainbow', label='Interferograms')
bridge_segments = LineCollection(bridges, color='black', linestyle='dashed', linewidths=1, label='Bridging Interferograms')
ax1.add_collection(line_segments)
ax1.add_collection(bridge_segments)
ax1.set(
    ylabel='Interferogram Number',
    xlabel='Date',
    xlim=(mdates.date2num(np.min(pair_list['date1'])) - 10, mdates.date2num(np.max(pair_list['date2'])) + 10),
    ylim=(-1, pair_list.shape[0] + 1),
)
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%b'))
ax1.legend(loc='upper left')

ax2.plot(date_range, coverage, color='black', linewidth=2)
ax2.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%b'))
ax2.set(xlabel='Date', ylabel='# of interferograms covering date', ylim=(0, np.max(coverage) + 1))

plt.tight_layout()

Each line in the top graph represents an interferogram. You can see that most interferograms span a relatively small date window, but that there are a few long temporal baseline interferograms that span the winter snow season. The bottom graph is a histogram displaying how many interferograms cover each date. As you can see, we have at least one interferogram for each date, so our network is fully connected!

##### 3.3.1.2 Network Quality

The next thing we want to consider is network quality. Setting an average coherence threshold of `0.7` already excludes a lot of interferograms (the interferograms whose numbers are red in the graph below are excluded), but there are still some we could consider removing to avoid including interferograms with unwrapping errors. For instance take a look at interferogram number 12 in the `ascending_50` stack.

In [ ]:
from mintpy.cli import view
view.main(f'ascending_50/inputs/ifgramStack.h5 --noverbose'.split())

If we wanted to remove this interferogram we could run MintPy's `modify_network.py` utility as shown below.

In [ ]:
# !modify_network.py {stack}/inputs/ifgramStack.h5 --exclude-ifg-index 12

Then, if we wanted to reset our network for any reason, we could use this command.

In [ ]:
# !modify_network.py {stack}/inputs/ifgramStack.h5 --reset

##### 3.3.1.3 View final network

Now that we've made all of our modifications, let's run `smallbaselineApp.py`'s `modify_network` step and view the results

In [ ]:
!smallbaselineApp.py --dir $stack $stack/mintpy_config.txt --dostep modify_network

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from mintpy.cli import plot_network

plot_network.main(f'{stack}/inputs/ifgramStack.h5 --figsize 12 3'.split())

#### 3.3.2 Check Reference Point

The next important thing to check is that we've selected an appropriate reference point. Since all deformation in our time series will be relative to this point, it is very important that this point is highly coherent and is in a non-deforming location. Note that if you are using multiple co-located stacks (like we are) you should use the same reference point for all stacks. First, we need to run the `reference_point` step:

In [ ]:
!smallbaselineApp.py --dir $stack $stack/mintpy_config.txt --dostep reference_point

Then we can view the location of the reference point on top of the average spatial coherence:

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
from mintpy.cli import view
from mintpy.utils import readfile

cfg = readfile.read_template(f'{stack}/smallbaselineApp.cfg')
ref_la, ref_lo = [int(coord) for coord in cfg['mintpy.reference.lalo'].strip("[]").split(', ')]
view.main(f'{stack}/avgSpatialCoh.h5 --pts-lalo {ref_la} {ref_lo} --noverbose --figsize 8 8'.split())

By zooming in on this plot, we can see that our reference point has a coherence value of 0.8 - that should be good enough.

#### 3.3.3 Estimate Velocity

Having checked that our network and reference point are good, we will run the remainder of the analysis. This includes performing the time series inversion, the tropospheric correction, and the topographic residual correction. If you want to read more about MintPy's SBAS workflow, you can read the paper describing the MintPy workflow [here](https://doi.org/10.1016/j.cageo.2019.104331).

In [ ]:
!smallbaselineApp.py --dir $stack $stack/mintpy_config.txt --start quick_overview

Congrats - you have successfully completed a time series InSAR analysis! Use the interactive plot below to view Mt. Edgecumbe's deformation history at specific locations.

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
from mintpy.cli import tsview

cmd = f'{stack}/timeseries_ERA5_demErr.h5 --figsize 9 3 --noverbose'
tsview.main(cmd.split())

**Remember to run MintPy for all four stacks before moving on!**

## 4. Data Combination

Now that you have **run MintPy for all four stacks**, let's take a look at the results.

### 4.1 Effect of Water Masking

The first thing that we will take a look at is a real problem that Grapenthin et al. had to troubleshoot when they were getting ready to publish their analysis. In the first version of their analysis, the water mask option HyP3 provided did not mask the water within 100 meters of the land surface. Even this relatively small amount of water present in the scene led to an area of erroneous subsidence northeast Mt. Edgecumbe. The HyP3 team has now tightened their water mask so that it follows the coastline closely, but we can re-create this issue by comparing the water masked and non water masked versions of our descending track stack.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from mintpy.utils import readfile

velocity_nomask, _ = readfile.read('descending_174_nomask/velocity.h5')
velocity_nomask = np.ma.masked_equal(velocity_nomask,0)

velocity_mask, _ = readfile.read('descending_174/velocity.h5')
velocity_mask = np.ma.masked_equal(velocity_mask,0)

difference = velocity_mask - velocity_nomask

f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize = (12, 6))
ax1.imshow(velocity_mask, cmap='jet', vmin=-0.04, vmax=0.07)
ax1.set(title='Water Masked')
ax2.imshow(velocity_nomask, cmap='jet', vmin=-0.04, vmax=0.07)
ax2.set(title='Not Water Masked')
diff_plot = ax3.imshow(difference, cmap='jet', norm=colors.CenteredNorm(halfrange=np.std(difference)))
ax3.set(title='Water Masked - Not Water Masked')

cax = ax3.inset_axes([1.04, 0.2, 0.05, 0.6])
f.colorbar(diff_plot, ax=ax3, cax=cax)

plt.setp(plt.gcf().get_axes(), xticks=[], yticks=[])
plt.tight_layout()

### 4.2 Final Figure

Finally, let's put it all together. Using the final velocity files from each stack (except the descending stack that did not have water masking applied), we can recreate the main portion of [Figure 2 from Grapenthin et al., 2022](https://doi.org/10.1029/2022GL099464).

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from mintpy.utils import readfile

ascending_50, _ = readfile.read('ascending_50/velocity.h5')
ascending_50 = np.ma.masked_equal(ascending_50, 0)

ascending_79, _ = readfile.read('ascending_79/velocity.h5')
ascending_79 = np.ma.masked_equal(ascending_79, 0)

descending_174, _ = readfile.read('descending_174/velocity.h5')
descending_174 = np.ma.masked_equal(descending_174, 0)

f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize = (12, 6))
ax1.imshow(ascending_79, cmap='jet', vmin=-0.04, vmax=0.07)
ax1.set(title='Ascending 79')
ax2.imshow(ascending_50, cmap='jet', vmin=-0.04, vmax=0.07)
ax2.set(title='Ascending 50')
cbar_plot = ax3.imshow(descending_174, cmap='jet', vmin=-0.04, vmax=0.07)
ax3.set(title='Descending 174')

cax = ax3.inset_axes([1.05, 0.25, 0.05, 0.5])
f.colorbar(cbar_plot, ax=ax3, cax=cax, orientation='vertical')

plt.setp(plt.gcf().get_axes(), xticks=[], yticks=[])
plt.tight_layout()

CONGRATULATIONS - you have just performed a publication-quality time series InSAR analysis!